# TDA_Candles

# DB

In [7]:
dct = {
    "F": {
        "market_value": 0
    }
}
json.dumps(dct)

'{"F": {"market_value": 0}}'

In [1]:
# Import required packages

from boto.s3.connection import S3Connection 
from deta import Deta
from dotenv import load_dotenv
import os
import streamlit as st

# Connect to Deta Base

def connect_db():
    if ".env" in os.listdir():
        env = load_dotenv(".env")
        DETA_KEY = os.getenv("DETA_KEY")
    else:
        try:
            DETA_KEY = os.environ["DETA_KEY"]
            # conn = S3Connection(DETA_KEY)
        except KeyError:
            DETA_KEY = st.secrets["DETA_KEY"]
    deta = Deta(DETA_KEY)
    return deta

2022-09-17 17:13:26.999 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


# DB Setup

In [2]:
# Import packages

from boto.s3.connection import S3Connection
import datetime as dt
# from db import *
from deta import Deta
from dotenv import load_dotenv
import json
import math
import numpy as np
import os
import pandas as pd
import plotly.express as px
import pytz
import requests
import streamlit as st
import streamlit_authenticator as stauth
from streamlit_option_menu import option_menu
import ta

In [69]:
# Connect to database

deta = connect_db()
config_db = deta.Base("config_db")
users_db = deta.Base("users_db")
tickers_db = deta.Base("tickers_db")

In [70]:
tickers_info = tickers_db.fetch().items
tickers = [item["key"].upper() for item in tickers_info]
ticker_dict = {}
for i in range(len(tickers)):
    ticker_dict[tickers[i]] = tickers_info[i]
    ticker_dict[tickers[i]]["option_symbol"] = tickers[i]
    ticker_dict[tickers[i]]["average_price"] = 0
    ticker_dict[tickers[i]]["market_value"] = 0
    ticker_dict[tickers[i]]["long_shares"] = 0
    ticker_dict[tickers[i]]["short_shares"] = 0
    ticker_dict[tickers[i]]["quantity"] = 0
    ticker_dict[tickers[i]]["option_last"] = 0

In [71]:
checkpoints_db = deta.Base("checkpoints_db")
checkpoints_info = checkpoints_db.fetch().items
for i in range(len(checkpoints_info)):
    key2 = checkpoints_info[i]['key']
    if key2 in tickers:
        idx = tickers.index(key2)
        for key, value in checkpoints_info[i].items():
            ticker_dict[tickers[idx]][key] = value

In [4]:
# Global variables

utc = pytz.timezone('UTC')
local_timezone = pytz.timezone('US/Pacific')
tda_base = 'https://api.tdameritrade.com/v1/'

In [5]:
# Get all items

items = tickers_db.fetch().items
items

[{'candle_type': 'Heikin Ashi',
  'confirm_time': 5,
  'confirm_unit': 'second',
  'contracts': 1,
  'delta_min': 50,
  'dte_min': 0,
  'ema_length': 100,
  'extended_hours': False,
  'frequency': 5,
  'frequency_type': 'minute',
  'hma_length': 7,
  'indicator': 'HMA',
  'key': 'F',
  'max': 0,
  'min': 0,
  'order_type': 'MARKET',
  'pause_time': 5,
  'pause_unit': 'minute',
  'period': 10,
  'period_type': 'day',
  'stoploss': 1000,
  'stoploss_pct': 10,
  'stoploss_type': 'Fixed $',
  'take_profit': 1000,
  'take_profit_pct': 100,
  'take_profit_type': 'Fixed $',
  'time_in_candle': 0,
  'trail_pct': 10,
  'trigger_trail': 100000,
  'trigger_trail_pct': 100,
  'trigger_trail_trail_pct': 100,
  'trigger_trail_type': 'Fixed $',
  'triggered': False,
  'use_close': False,
  'wick_requirement': 'Any wick okay'},
 {'candle_type': 'Heikin Ashi',
  'confirm_time': 5,
  'confirm_unit': 'second',
  'contracts': 1,
  'delta_min': 50,
  'dte_min': 0,
  'ema_length': 100,
  'extended_hours': F

In [7]:
# Delete all items

# keys = [item['key'] for item in items]
# for key in keys:
#     tickers_db.delete(key)

In [9]:
# Setup config database

# config_keys = ["HEROKU_API", "TDA_API", "TDA_REFRESH", "TDA_ACCOUNT"]
# config_values = [os.getenv(key) for key in config_keys]
# config_keys = config_keys + ["TDA_ACCESS", "TDA_LAST_ACCESS", "TDA_LAST_REFRESH", "TDA_ACCESS_LIMIT", "BOT_ON"]
# config_values = config_values + ["asfdasdf", "8/8/2022 20:54:30", "8/18/2022 19:08:26", "30", True]
# for i in range(len(config_keys)):
#     entry = {
#         "key": config_keys[i],
#         "value": config_values[i]
#     }
#     config_db.put(entry)

In [8]:
# # Setup tickers database

# tickers = ["SWN", "SNAP", "WBD", "CCL", "F", "HOOD"]
# period_types = ["day"] * len(tickers)
# periods = [10] * len(tickers)
# frequency_types = ["minute"] * len(tickers)
# frequencies = [30] * len(tickers)
# extended_hours = [False] * len(tickers)
# ema_lengths = [10] * len(tickers)
# hma_lengths = [7] * len(tickers)
# contracts = [1] * len(tickers)
# stoplosses = [1000] * len(tickers)
# stoploss_pcts = [10] * len(tickers)
# trailing_pcts = [10] * len(tickers)
# use_trailings = [False] * len(tickers)
# use_pcts = [False] * len(tickers)
# dte_mins = [0] * len(tickers)
# delta_mins = [50] * len(tickers)

# for i in range(len(tickers)):
#     entry = {
#         "key": tickers[i], 
#         "period_type": period_types[i], 
#         "period": periods[i],
#         "frequency_type": frequency_types[i],
#         "frequency": frequencies[i],
#         "extended_hours": extended_hours[i],
#         "ema_length": ema_lengths[i],
#         "hma_length": hma_lengths[i],
#         "contracts": contracts[i],
#         "stoploss": stoplosses[i],
#         "stoploss_pct": stoploss_pcts[i],
#         "trailing_pct": trailing_pcts[i],
#         "use_trailing": use_trailings[i],
#         "use_pct": use_pcts[i],
#         "dte_min": dte_mins[i],
#         "delta_min": delta_mins[i],
#     }
#     tickers_db.put(entry)

# TDA Functions

In [13]:
# Import packages

import datetime as dt
from deta import Deta
from dotenv import load_dotenv
import json
import numpy as np
import os
import pandas as pd
import pytz
import requests

# Database connection

def connect_db():
    load_dotenv(".env")
    DETA_KEY = os.getenv("DETA_KEY")
    deta = Deta(DETA_KEY)
    return deta

config_db = connect_db().Base("config_db")

# Global variables

utc = pytz.timezone('UTC')
local_timezone = pytz.timezone('US/Pacific')
tda_base = 'https://api.tdameritrade.com'
tda_access_limit = int(config_db.get("TDA_ACCESS_LIMIT")['value'])
tda_refresh = config_db.get("TDA_REFRESH")['value']
tda_api = config_db.get("TDA_API")['value']
tda_account = config_db.get("TDA_ACCOUNT")['value']

# TDA functions

def tda_authenticate():
    config_db = connect_db().Base("config_db")
    access_time_adj = int(tda_access_limit * 0.9)
    tda_access_last = pd.Timestamp(config_db.get("TDA_LAST_ACCESS")['value'], tz=utc).astimezone(local_timezone)
    tda_payload = {
        'grant_type': 'refresh_token',
        'refresh_token': tda_refresh,
        'client_id': tda_api,
    }
    tda_auth_url = '{}/v1/oauth2/token'.format(tda_base)
    now_time = dt.datetime.now(tz=local_timezone)
    minutes_since_refresh = round((now_time - tda_access_last).total_seconds() / 60, 2)
    if minutes_since_refresh > access_time_adj:
        tda_token_request = requests.post(tda_auth_url, data = tda_payload)
        tda_token_response = json.loads(tda_token_request.content)
        tda_access_token = tda_token_response['access_token']
        put_keys = ["TDA_ACCESS", "TDA_LAST_ACCESS"]
        put_values = [tda_access_token, dt.datetime.now(tz=utc).strftime("%Y-%m-%d %X")]
        for i in range(len(put_keys)):
            entry = {
                "key": put_keys[i],
                "value": put_values[i]
            }
            config_db.put(entry)
    else:
        tda_access_token = config_db.get("TDA_ACCESS")['value']
    tda_headers = {'Authorization': 'Bearer {}'.format(tda_access_token)}
    return tda_headers

def get_quote_tda(symbol="SPY"):
    quote_url = f"{tda_base}/v1/marketdata/quotes?apikey={tda_api}&symbol={symbol}"
    tda_headers = tda_authenticate()
    req = requests.get(quote_url, headers = tda_headers)
    resp = json.loads(req.content)
    return resp

def get_data_tda(ticker="SPY", periodType="day", period=10, frequencyType="minute", frequency=30, extended_hours=False):
    periodTypes = ["day", "month", "year", "ytd"]
    period_day = [1, 2, 3, 4, 5, 10]
    period_month = [1, 2, 3, 6]
    period_year = [1, 2, 3, 5, 10, 15, 20]
    period_ytd = [1]
    frequencyType_day = ["minute"]
    frequencyType_month = ["daily", "weekly"]
    frequencyType_year = ["daily", "weekly", "monthly"]
    frequencyType_ytd = ["daily", "weekly"]
    frequency_minute = [1, 5, 10, 15, 30]
    frequency_daily = [1]
    frequency_weekly = [1]
    frequency_monthly = [1]
    # ext = False
    useEpoch = False
    now = dt.datetime.now()
    epoch = dt.datetime.utcfromtimestamp(0)
    # now = dt.datetime.now(tz=utc)
    # epoch = pd.Timestamp(dt.datetime.utcfromtimestamp(0),tz=utc)
    epoch_now_diff = now - epoch
    epoch_to_now = epoch_now_diff.days * 24 * 60 * 60 * 1000 + (epoch_now_diff.seconds * 1000) + (int(epoch_now_diff.microseconds / 1000))
    then = now - dt.timedelta(days=1)
    epoch_then_diff = then - epoch
    epoch_to_then = epoch_then_diff.days * 24 * 60 * 60 * 1000 + (epoch_then_diff.seconds * 1000) + (int(epoch_then_diff.microseconds / 1000))
    startDate = epoch_to_then
    endDate = epoch_to_now
    data_url = f"{tda_base}/v1/marketdata/{ticker}/pricehistory?apikey={tda_api}&frequencyType={frequencyType}&frequency={frequency}&needExtendedHoursData={extended_hours}&endDate={endDate}"
    if useEpoch:
        data_url = data_url + f"&startDate={startDate}"
    else:
        data_url = data_url + f"&periodType={periodType}&period={period}"
    tda_headers = tda_authenticate()
    req = requests.get(data_url, headers = tda_headers)
    resp = json.loads(req.content)
    bars = resp['candles']
    if bars[-1]['close'] == 0:
        bars = bars[:-1]
    return bars

def get_positions_tda():
    tda_headers = tda_authenticate()
    tda_account_url = '{}/v1/accounts/{}'.format(tda_base, tda_account)
    tda_positions_url = '{}?fields=positions'.format(tda_account_url)
    tda_positions_request = requests.get(tda_positions_url, headers = tda_headers)
    tda_positions_content = json.loads(tda_positions_request.content)
    positions = []
    if 'positions' in tda_positions_content['securitiesAccount']:
        positions = tda_positions_content['securitiesAccount']['positions']
    else:
        positions = [{
            "instrument": {
                "symbol": "None"
            },
            "longQuantity": 0,
            "shortQuantity": 0,
            "averagePrice": 0,
            "marketValue": 0,
            "currentDayProfitLossPercentage": 0
        }]
    return positions

def get_orders_tda():
    tda_headers = tda_authenticate()
    tda_history_url = '{}/v1/accounts/{}/transactions'.format(tda_base, tda_account)
    tda_history_request = requests.get(tda_history_url, headers = tda_headers)
    tda_history_content = json.loads(tda_history_request.content)
    orders = []
    if len(tda_history_content) < 1:
        orders = [{
            "orderDate": "1/1/2000 12:00:00",
            "transactionItem": {
                "instrument": {
                    "symbol": "none"
                },
                "instruction": "none",
                "amount": 0,
                "price": 0
            }
        }]
    else:
        orders = [item for item in tda_history_content if item['type'] == 'TRADE']
    return orders

def get_orders2_tda():
    tda_headers = tda_authenticate()
    tda_account_url = '{}/v1/accounts/{}'.format(tda_base, tda_account)
    tda_orders_url = '{}?fields=orders'.format(tda_account_url)
    tda_orders_request = requests.get(tda_orders_url, headers = tda_headers)
    tda_orders_content = json.loads(tda_orders_request.content)
    print(tda_orders_content)
    orders = []
    if 'orderStrategies' in tda_orders_content['securitiesAccount']:
        orders = tda_orders_content['securitiesAccount']['orderStrategies']
    return orders

def get_hours_tda():
    tda_headers = tda_authenticate()
    # market_date = dt.datetime.now().strftime('%Y-%m-%d')
    # tda_hours_url = '{}/v1/marketdata/EQUITY/hours?apikey={}&date={}'.format(tda_base, tda_api, market_date)
    # tda_hours_content = json.loads(requests.get(tda_hours_url, headers = tda_headers).content)
    # market_open = pd.to_datetime(tda_hours_content['equity']['EQ']['sessionHours']['regularMarket'][0]['start'])
    # market_close = pd.to_datetime(tda_hours_content['equity']['EQ']['sessionHours']['regularMarket'][0]['end'])
    # if market_close > dt.datetime.now(tz=local_timezone) > market_open:
    #     market_is_open = True
    # else:
    #     market_is_open = False
    # return market_is_open
    current_time = dt.datetime.now(tz=local_timezone)
    market = "EQUITY"
    tda_hours_url = '{}/v1/marketdata/{}/hours?apikey={}'.format(tda_base, market, tda_api)
    tda_hours_request = requests.get(tda_hours_url, headers = tda_headers)
    if tda_hours_request.status_code != 200:
        tda_hours_content = tda_hours_request.content
        print(tda_hours_content)
        closing_time = current_time
        market_is_open = False
    else:
        tda_hours_content = json.loads(tda_hours_request.content)
        if 'EQ' in list(tda_hours_content['equity'].keys()):
            closing_time = pd.to_datetime(tda_hours_content['equity']['EQ']['sessionHours']['regularMarket'][0]['end']).astimezone(local_timezone)
            market_is_open = tda_hours_content['equity']['EQ']['isOpen']
        else:
            closing_time = current_time
            market_is_open = False
    return (market_is_open, closing_time)

def get_chain_tda(ticker):
    result = {}
    contract_type = "ALL" # ALL (default), CALL, PUT
    delta_min = int(tickers_db.get(ticker)['delta_min'])
    # delta_min = 90
    strike_count = 3
    from50 = np.round(abs(1 - delta_min / 50), 2)
    while from50 > 0:
        strike_count += 2
        from50 = np.round(from50 - 0.05, 2)
    moneyness = "ALL" # ITM, NTM, OTM, SAK, SBK, SNK, ALL
    now = dt.datetime.now()
    dte_min = int(tickers_db.get(ticker)['dte_min'])
    from_date = (now - dt.timedelta(days=dte_min)).strftime("%Y-%m-%d")
    to_date = (now + dt.timedelta(days=dte_min+32)).strftime("%Y-%m-%d")
    chain_url = f'{tda_base}/v1/marketdata/chains?apikey={tda_api}&symbol={ticker}&contractType={contract_type} \
                &includeQuotes=false&strikeCount={strike_count}&range={moneyness}&fromDate={from_date}&toDate={to_date} \
                &optionType=S'
    tda_headers = tda_authenticate()
    tda_chain_request = requests.get(chain_url, headers = tda_headers)
    if tda_chain_request.status_code != 200:
        print("Error: TDA status code")
        tda_chain_content = tda_chain_request.content
    else:
        tda_chain_content = json.loads(tda_chain_request.content)
        if tda_chain_content['status'] != 'FAILED':
            calls = tda_chain_content['callExpDateMap']
            call_exps = list(calls.keys())
            call_exp = call_exps[0]
            call_strikes = list(calls[call_exp].keys())
            call_strikes = list(reversed(call_strikes))
            call_strike = 0
            i = 0
            while i < len(call_strikes):
                call = calls[call_exp][call_strikes[i]][0]
                call_delta = np.round(abs(float(call['delta'])) * 100)
                if call_delta > delta_min:
                    # print(call_delta)
                    call_strike = call_strikes[i]
                    break
                i += 1
            puts = tda_chain_content['putExpDateMap']
            put_exps = list(puts.keys())
            put_exp = put_exps[0]
            put_strikes = list(puts[put_exp].keys())
            put_strike = 0
            i = 0
            while i < len(put_strikes):
                put = puts[put_exp][put_strikes[i]][0]
                put_delta = np.round(abs(float(put['delta'])) * 100)
                if put_delta > delta_min:
                    # print(put_delta)
                    put_strike = put_strikes[i]
                    break
                i += 1
            
    result = {
        "put": put['symbol'],
        "call": call['symbol']
    }
    return result

def tda_submit_order(instruction, quantity, symbol, assetType="OPTION"):
    tda_orders_url = '{}/v1/accounts/{}/orders'.format(tda_base, tda_account)
    if "_" not in symbol:
        assetType = "EQUITY"
        if instruction == "SELL_TO_OPEN":
            instruction = "SELL_SHORT"
        elif instruction == "BUY_TO_CLOSE":
            instruction = "BUY_TO_COVER"
        elif instruction == "BUY_TO_OPEN":
            instruction = "BUY"
        elif instruction == "SELL_TO_CLOSE":
            instruction = "SELL"
    data = {
        "orderType": "MARKET",
        "session": "NORMAL",
        "duration": "DAY",
        "orderStrategyType": "SINGLE",
        "orderLegCollection": [
            {
            "instruction": instruction,
            "quantity": quantity,
            "instrument": {
                "symbol": symbol,
                "assetType": assetType # EQUITY or OPTION
            }
            }
        ]
    }
    tda_headers = tda_authenticate()
    r = requests.post(tda_orders_url, json=data, headers = tda_headers)
    if r.status_code in [200, 201]:
        print(f"{r.status_code}: {r.content}")
        return r.content
    else:
        print(f"{r.status_code}: {json.loads(r.content)}")
        return json.loads(r.content)

# Script

In [11]:
# This is the main file that executes the automated trades. Everything else is a supporting document.

# Import required packages

import datetime as dt
# from db import *
import math
import numpy as np
import pandas as pd
import ta
# from tda import *
import time

In [169]:
# Global variables

log_count = 0

In [170]:
# Define the strategy

#def run():

    # Execution start

time_start = time.time()

In [171]:
# Connect to Deta

deta = connect_db()
config_db = deta.Base("config_db")
tda_account = config_db.get("TDA_ACCOUNT")['value']

tickers_db = deta.Base("tickers_db")
tickers_info = tickers_db.fetch().items
tickers = [item["key"].upper() for item in tickers_info]
period_types = [item["period_type"] for item in tickers_info]
periods = [item["period"] for item in tickers_info]
frequencies = [item["frequency"] for item in tickers_info]
frequency_types = [item["frequency_type"] for item in tickers_info]
extended_hours = [item["extended_hours"] for item in tickers_info]
ema_lengths = [item["ema_length"] for item in tickers_info]
hma_lengths = [item["hma_length"] for item in tickers_info]
contracts = [item["contracts"] for item in tickers_info]
stoplosses = [item["stoploss"] for item in tickers_info]
stoploss_pcts = [item["stoploss_pct"] for item in tickers_info]
trailing_pcts = [item["trailing_pct"] for item in tickers_info]
use_pcts = [item["use_pct"] for item in tickers_info]
use_trailings = [item["use_trailing"] for item in tickers_info]

In [172]:
# Get market status

hours = get_hours_tda()
market_open = hours[0]
closing_time = hours[1]
#current_time = hours[2]

In [173]:
# Examine watchlist

lasts, bullish_candles, emas, hmas, bullish_hmas, chains = [], [], [], [], [], []
for ticker in tickers:
    ticker = "WBD"

In [174]:
# Get quotes

quote = get_quote_tda(ticker)
last = np.round(float(quote[ticker]['lastPrice']), 2)
lasts.append(last)

In [175]:
# Fetch ticker data settings from Deta

idx = tickers.index(ticker)
periodType = tickers_info[idx]['period_type']
period = int(tickers_info[idx]['period'])
frequencyType = tickers_info[idx]['frequency_type']
frequency = int(tickers_info[idx]['frequency'])
extended_hours = tickers_info[idx]['extended_hours']

In [176]:
# Fetch data from TDA API

data = get_data_tda(ticker=ticker, periodType=periodType, period=period, frequencyType=frequencyType, frequency=frequency, extended_hours=extended_hours)
tda_opens = [item['open'] for item in data]
tda_highs = [item['high'] for item in data]
tda_lows = [item['low'] for item in data]
tda_closes = [item['close'] for item in data]

{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close': 0.0, 'volume': 0, 'datetime': 1661425200000}


In [177]:
# Change to Heikin Ashi candles

ha_opens = [None] * len(tda_opens)
ha_highs = [None] * len(tda_highs)
ha_lows = [None] * len(tda_lows)
ha_closes = [None] * len(tda_closes)
ha_closes[0] = 0.25 * (tda_opens[0] + tda_highs[0] + tda_lows[0] + tda_closes[0])
ha_opens[0] = tda_opens[0]
ha_highs[0] = max(tda_highs[0], ha_opens[0], ha_closes[0])
ha_lows[0] = min(tda_lows[0], ha_opens[0], ha_closes[0])
i = 1
while i < len(ha_closes):
    ha_opens[i] = 0.5 * (ha_opens[i-1] + ha_closes[i-1])
    ha_highs[i] = max(tda_highs[i], ha_opens[i], tda_closes[i])
    ha_lows[i] = min(tda_lows[i], ha_opens[i], tda_closes[i])
    ha_closes[i] = 0.25 * (ha_opens[i] + tda_highs[i] + tda_lows[i] + tda_closes[i])
    i += 1
tda_opens = ha_opens
tda_highs = ha_highs
tda_lows = ha_lows
tda_closes = ha_closes

In [178]:
# Bullish or bearish candle

if market_open:
    last_close = tda_closes[len(tda_closes)-2]
    last_open = tda_opens[len(tda_opens)-2]
else:
    last_close = tda_closes[len(tda_closes)-1]
    last_open = tda_opens[len(tda_opens)-1]
if last_close > last_open:
    bullish_candle = True
else:
    bullish_candle = False
bullish_candles.append(bullish_candle)

In [193]:
# Create dataframe

df = pd.DataFrame()
if frequencyType == "minute":
    time_format = "%m/%d %H:%M"
elif frequencyType == "daily":
    time_format = "%m/%d"
elif frequencyType == "weekly":
    time_format = "%m/%d/%Y"
elif frequencyType == "monthly":
    time_format = "%b %Y"
tda_times = [candle['datetime'] for candle in data]
tda_dates = [dt.datetime.utcfromtimestamp(time_ / 1000) for time_ in tda_times]
tda_locals = [utc.localize(date).astimezone(local_timezone) for date in tda_dates]
tda_datetimes = [time_.strftime(time_format) for time_ in tda_locals]
tda_datetimes = [""] + tda_datetimes
tda_opens = [""] + tda_opens
tda_highs = [""] + tda_highs
tda_lows = [""] + tda_lows
tda_closes = [""] + tda_closes
df['datetime'] = tda_datetimes
df['open'] = tda_opens
df['high'] = tda_highs
df['low'] = tda_lows
df['close'] = tda_closes

In [194]:
# Calculate indicators

ema_window = tickers_info[idx]['ema_length']
emas_all = np.round(ta.trend.ema_indicator(pd.to_numeric(df['close']), window=ema_window), 2)
ema = emas_all[len(emas_all)-1]
emas.append(ema)

hma_window = tickers_info[idx]['hma_length']
hma1 = ta.trend.wma_indicator(pd.to_numeric(df['close']), window=int(hma_window/2))
hma2 = ta.trend.wma_indicator(pd.to_numeric(df['close']), window=hma_window)
hma_raw = (2 * hma1) - hma2
hmas_all = np.round(ta.trend.wma_indicator(hma_raw, window=math.floor(math.sqrt(hma_window))), 2)
hma = hmas_all[len(hmas_all)-1]
hmas.append(hma)

In [195]:
# Bullish or bearish HMA

if market_open:
    last_hma = hmas_all[len(hmas_all)-2]
    last_hma1 = hmas_all[len(hmas_all)-3]
else:
    last_hma = hmas_all[len(hmas_all)-1]
    last_hma1 = hmas_all[len(hmas_all)-1]
if last_hma > last_hma1:
    bullish_hma = True
else:
    bullish_hma = False
bullish_hmas.append(bullish_hma)

In [196]:
# Get option symbols

chain = get_chain_tda(ticker)
chains.append(chain)

95.0
94.0


In [197]:
# Get positions

tda_positions = get_positions_tda()
if tda_positions != []:
    tda_tickers_held = [position['instrument']['symbol'] for position in tda_positions]
    tda_average_prices = [np.round(float(position['averagePrice']),2) for position in tda_positions]
    tda_market_values = [np.round(float(position['marketValue']),2) for position in tda_positions]
    tda_long_shares = [int(position['longQuantity']) for position in tda_positions]
    tda_short_shares = [int(position['shortQuantity']) for position in tda_positions]
    tda_quantities = list(np.array(tda_long_shares) - np.array(tda_short_shares))
else:
    tda_tickers_held = []
    tda_average_prices = []
    tda_market_values = []
    tda_long_shares = []
    tda_short_shares = []
    tda_quantities = []

In [198]:
# Adjust tickers for optional holdings

tda_symbols_held = []
for ticker in tda_tickers_held:
    symb = ticker
    if "_" in ticker:
        symb = ticker.split("_")[0]
    tda_symbols_held.append(symb)
held_symbol_dict = dict(zip(tda_symbols_held, tda_tickers_held))

In [199]:
# Calculate gainloss from quotes and average prices

tda_gainlosses, tda_costbases, tda_gainloss_pcts = [], [], []
for ticker in tickers:
    if ticker in tda_symbols_held:
        idx = tda_symbols_held.index(ticker)
        if tda_long_shares[idx] > 0:
            tda_costbasis = np.round(tda_average_prices[idx] * tda_long_shares[idx], 2)
            tda_gainloss = np.round(tda_market_values[idx] - (tda_average_prices[idx] * tda_long_shares[idx]), 2)
            tda_gainloss_pct = np.round(tda_gainloss / tda_costbasis * 100, 2)
        elif tda_short_shares[idx] > 0:
            tda_costbasis = np.round(tda_average_prices[idx] * tda_short_shares[idx], 2)
            tda_gainloss = np.round(abs(tda_average_prices[idx] * tda_short_shares[idx]) - abs(tda_market_values[idx]), 2)
            tda_gainloss_pct = np.round(tda_gainloss / tda_costbasis * 100, 2)
        else:
            tda_costbasis = 0
            tda_gainloss = 0
            tda_gainloss_pct = 0
    else:
        tda_costbasis = 0
        tda_gainloss = 0
        tda_gainloss_pct = 0
    tda_gainlosses.append(tda_gainloss)
    tda_costbases.append(tda_costbasis)
    tda_gainloss_pcts.append(tda_gainloss_pct)

In [200]:
# Map watchlist data to positions data

ticker_quantity_dict = dict(zip(tda_tickers_held, tda_quantities))
ticker_gainloss_dict = dict(zip(tickers, tda_gainlosses))
ticker_gainloss_pct_dict = dict(zip(tickers, tda_gainloss_pcts))
stoploss_dict, stoploss_pct_dict, trailing_pct_dict, use_pct_dict, use_trailing_dict = {}, {}, {}, {}, {}
for i in range(len(tickers)):
    if tickers[i] in tda_symbols_held:
        idx = tda_symbols_held.index(tickers[i])
        # costbasis = tda_costbases[idx]
        stoploss = tickers_db[idx]['stoploss']
        stoploss_pct = tickers_db[idx]['stoploss_pct']
        trailing_pct = tickers_db[idx]['trailing_pct']
        use_pct = tickers_db[idx]['use_pct']
        use_trailing = tickers_db[idx]['use_trailing']
        # max_loss_adj = np.round(costbasis * max_loss / 100, 2)
        stoploss_dict[tickers[i]] = stoploss
        stoploss_pct_dict[tickers[i]] = stoploss_pct
        trailing_pct_dict[tickers[i]] = trailing_pct
        use_pct_dict[tickers[i]] = use_pct
        use_trailing_dict[tickers[i]] = use_trailing

In [201]:
# Make entry trades

current_time = dt.datetime.now(tz=local_timezone)
minutes_until_close = np.round((closing_time - current_time).total_seconds() / 60, 2)
wl_tickers_held = list(set(tickers).intersection(tda_symbols_held))
ticker_long_dict = dict(zip(tda_tickers_held, tda_long_shares))
ticker_short_dict = dict(zip(tda_tickers_held, tda_short_shares))
ticker_gainloss_dict = dict(zip(tickers, tda_gainlosses))

In [202]:
# Entry order

for i in range(len(bullish_candles)):
    # condition1 = market_open
    condition1 = 0 < minutes_until_close < 450
    condition2 = tickers[i] not in tda_symbols_held
    condition3 = bullish_candles[i]
    condition4 = bullish_hmas[i]
    bullish_entry = condition1 and condition2 and condition3 and condition4
    bearish_entry = condition1 and condition2 and not condition3 and not condition4
    if bullish_entry:
        entry_symbol = chains[i]['call']
        # entry_order = tda_submit_order("BUY_TO_OPEN", contracts[i], entry_symbol)
        print(f"Bullish entry: Buying {contracts[i]} contracts of {entry_symbol}")
    if bearish_entry:
        entry_symbol = chains[i]['put']
        # entry_order = tda_submit_order("BUY_TO_OPEN", contracts[i], entry_symbol)
        print(f"Bearish entry: Buying {contracts[i]} contracts of {entry_symbol}")

In [203]:
# Exit order

for i in range(len(bullish_candles)):
    # condition1 = market_open
    condition1 = 0 < minutes_until_close < 450
    condition2 = tickers[i] in tda_symbols_held
    if condition2:
        if use_pcts[i]:
            pct_limit = np.negative(stoploss_pct_dict[tickers[i]])
            pct_real = ticker_gainloss_pct_dict[tickers[i]]
            condition3 = pct_limit > pct_real
        else:
            stop_limit = np.negative(stoploss_dict[tickers[i]])
            stop_real = ticker_gainloss_dict[tickers[i]]
            condition3 = stop_limit > stop_real
        exit_symbol = held_symbol_dict[tickers[i]]
        exit_quantity = ticker_quantity_dict[tickers[i]]
        stoploss_exit = condition1 and condition2 and condition3
        condition4 = "C" in exit_symbol.split("_")[1]
        condition5 = bullish_candles[i]
        call_exit = condition1 and condition2 and condition4 and not condition5
        put_exit = condition1 and condition2 and not condition4 and condition5
        if stoploss_exit or call_exit or put_exit:
            # time.sleep(5)
            # exit_order = tda_submit_order("SELL_TO_CLOSE", exit_quantity, exit_symbol)
            exit_log = f"Closing out {exit_quantity} contracts of {exit_symbol} due to"
            if stoploss_exit:
                exit_log = exit_log + f" stop loss {stop_limit} > {stop_real}"
            elif call_exit:
                exit_log = exit_log + f" bearish candle while holding call"
            elif put_exit:
                exit_log = exit_log + f" bullish candle while holding put"
            print(exit_log)

In [204]:
# Print log

now = dt.datetime.now(tz=local_timezone).strftime("%Y-%m-%d %X")
time_end = time.time()
time_total = np.round(time_end - time_start, 2)
# global log_count
log_count += 1
if log_count % 1 == 0:
    print_log = [
        ["Time", f" = {now}"],
        ["Market open", f" = {market_open}"],
        ["Held Tickers", f" = {tda_tickers_held}"],
        ["Long shares",f" = {tda_long_shares}"],
        ["Short shares",f" = {tda_short_shares}"],
        ["Quantities Held",f" = {tda_quantities}"],
        ["Average prices",f" = {tda_average_prices}"],
        ["Market values",f" = {tda_market_values}"],
        ["Number Tickers", f" = {len(tickers)}"],
        ["Tickers",f" = {tickers}"],
        ["Contracts",f" = {contracts}"],
        ["Stoplosses",f" = {stoplosses}"],
        ["Stoploss pcts",f" = {stoploss_pcts}"],
        ["Trailing pcts",f" = {trailing_pcts}"],
        ["Use pct?",f" = {use_pcts}"],
        ["Use trailing?",f" = {use_trailings}"],
        ["Period types",f" = {period_types}"],
        ["Periods",f" = {periods}"],
        ["Frequency types",f" = {frequency_types}"],
        ["Frequencies",f" = {frequencies}"],
        ["EMA lengths",f" = {ema_lengths}"],
        ["HMA lengths",f" = {hma_lengths}"],
        ["EMAs",f" = {emas}"],
        ["HMAs",f" = {hmas}"],
        ["Extended hours",f" = {extended_hours}"],
        ["Lasts",f" = {lasts}"],
        ["Bullish Candles",f" = {bullish_candles}"],
        ["Bullish Candles",f" = {bullish_hmas}"],
        ["Execution Speed",f" = {time_total} seconds"],
        ["---------------", "---------------"]
    ]
else:
    print_log = [
        ["No"," log"],
        ["---------------", "---------------"]
    ]

for item_list in print_log:
    for i in item_list:
        print(i.ljust(20), end='')
    print()

Time                 = 2022-08-26 02:46:31
Market open          = True             
Held Tickers         = ['None']         
Long shares          = [0]              
Short shares         = [0]              
Quantities Held      = [0]              
Average prices       = [0.0]            
Market values        = [0.0]            
Number Tickers       = 6                
Tickers              = ['CCL', 'F', 'HOOD', 'SNAP', 'SWN', 'WBD']
Contracts            = [1, 1, 1, 1, 1, 1]
Stoplosses           = [1000, 1000, 1000, 1000, 1000, 1000]
Stoploss pcts        = [10, 10, 10, 10, 10, 10]
Trailing pcts        = [10, 10, 10, 10, 10, 10]
Use pct?             = [False, False, False, False, False, False]
Use trailing?        = [False, False, False, False, False, False]
Period types         = ['day', 'day', 'day', 'day', 'day', 'day']
Periods              = [10, 10, 10, 10, 10, 10]
Frequency types      = ['minute', 'minute', 'minute', 'minute', 'minute', 'minute']
Frequencies          = [30, 30, 30,

In [205]:
import plotly.graph_objs as go
selected_indicators = ["EMA", "HMA"]
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df['datetime'].append(pd.Series(" ")),
    open = df['open'].append(pd.Series(" ")),
    high = df['high'].append(pd.Series(" ")),
    low = df['low'].append(pd.Series(" ")),
    close = df['close'].append(pd.Series(" ")), 
    name = 'Candles'
))
if "EMA" in selected_indicators:
    ema_window = tickers_info[idx]['ema_length']
    ema = np.round(ta.trend.ema_indicator(pd.to_numeric(df['close']), window=ema_window), 2)
    fig.add_trace(go.Scatter(
        x = df['datetime'], 
        y = ema,
        line = dict(color='blue', width=1.5), 
        name = f'EMA {ema_window}'
    ))
if "HMA" in selected_indicators:
    hma_window = tickers_info[idx]['hma_length']
    hma1 = ta.trend.wma_indicator(pd.to_numeric(df['close']), window=int(hma_window/2))
    hma2 = ta.trend.wma_indicator(pd.to_numeric(df['close']), window=hma_window)
    hma_raw = (2 * hma1) - hma2
    hma = np.round(ta.trend.wma_indicator(hma_raw, window=math.floor(math.sqrt(hma_window))), 2)
    fig.add_trace(go.Scatter(
        x = df['datetime'], 
        y = hma,
        line = dict(color='purple', width=1.5), 
        name = f'HMA {hma_window}'
    ))
fig.update_layout(
    title = f'Chart: {ticker} ({last}), {frequency} {frequencyType}',
    height = 700,
    yaxis_title = 'Price',
    xaxis_title = 'Datetime',
    plot_bgcolor = 'gainsboro'
)
fig.update_xaxes(
    rangeslider_visible=False,
    showgrid=False,
    tickprefix=" "
)
fig.update_yaxes(
    showgrid=False,
    ticksuffix=" "
)
fig.show()

In [39]:
def get_data_tda(ticker="SPY", periodType="day", period=10, frequencyType="minute", frequency=30, extended_hours=False):
    periodTypes = ["day", "month", "year", "ytd"]
    period_day = [1, 2, 3, 4, 5, 10]
    period_month = [1, 2, 3, 6]
    period_year = [1, 2, 3, 5, 10, 15, 20]
    period_ytd = [1]
    frequencyType_day = ["minute"]
    frequencyType_month = ["daily", "weekly"]
    frequencyType_year = ["daily", "weekly", "monthly"]
    frequencyType_ytd = ["daily", "weekly"]
    frequency_minute = [1, 5, 10, 15, 30]
    frequency_daily = [1]
    frequency_weekly = [1]
    frequency_monthly = [1]
    factor = 1
    if frequencyType in ["daily", "weekly", "monthly"] and frequency != 1:
        factor = frequency
        frequency = 1
    if frequencyType=="hour":
        frequencyType = "minute"
        frequency = int(frequency * 60)
    if frequencyType == "minute" and frequency not in frequency_minute:
        freq_min_rev = list(reversed(frequency_minute))
        i = 0
        while i < len(freq_min_rev):
            if frequency % freq_min_rev[i] == 0:
                factor = int(frequency / freq_min_rev[i])
                frequency = freq_min_rev[i]
                break
            i += 1
    # ext = False
    useEpoch = False
    now = dt.datetime.now()
    epoch = dt.datetime.utcfromtimestamp(0)
    # now = dt.datetime.now(tz=utc)
    # epoch = pd.Timestamp(dt.datetime.utcfromtimestamp(0),tz=utc)
    epoch_now_diff = now - epoch
    epoch_to_now = epoch_now_diff.days * 24 * 60 * 60 * 1000 + (epoch_now_diff.seconds * 1000) + (int(epoch_now_diff.microseconds / 1000))
    then = now - dt.timedelta(days=1)
    epoch_then_diff = then - epoch
    epoch_to_then = epoch_then_diff.days * 24 * 60 * 60 * 1000 + (epoch_then_diff.seconds * 1000) + (int(epoch_then_diff.microseconds / 1000))
    startDate = epoch_to_then
    endDate = epoch_to_now
    data_url = f"{tda_base}/v1/marketdata/{ticker}/pricehistory?apikey={tda_api}&frequencyType={frequencyType}&frequency={frequency}&needExtendedHoursData={extended_hours}&endDate={endDate}"
    if useEpoch:
        data_url = data_url + f"&startDate={startDate}"
    else:
        data_url = data_url + f"&periodType={periodType}&period={period}"
    print(data_url)
    tda_headers = tda_authenticate()
    req = requests.get(data_url, headers = tda_headers)
    resp = json.loads(req.content)
    if 'candles' not in resp:
        print(resp)
    bars = resp['candles']
    if bars[-1]['close'] == 0:
        bars = bars[:-1]
    if factor != 1:
        if frequencyType == "minute" and frequency == 30:
            if factor == 2:
                bar_hour_starts_utc = ["13:30", "14:30", "15:30", "16:30", "17:30", "18:30", "19:30"]
            elif factor == 4:
                bar_hour_starts_utc = ["13:30", "15:30", "17:30", "19:30"]
            elif factor == 6:
                bar_hour_starts_utc = ["13:30", "16:30", "19:30"]
            elif factor == 8:
                bar_hour_starts_utc = ["13:30", "17:30"]
            elif factor == 10:
                bar_hour_starts_utc = ["13:30", "18:30"]
            elif factor == 12:
                bar_hour_starts_utc = ["13:30", "19:30"]
            bars_hour = []
            bars_Xmin = []
            highs_Xmin = []
            lows_Xmin = []
            for bar in bars:
                timestampX = pd.to_datetime(int(bar['datetime']) * 1000000)
                hourX = timestampX.strftime("%H:%M")
                if hourX in bar_hour_starts_utc and len(bars_Xmin) > 0:
                    highs_Xmin = []
                    lows_Xmin = []
                    for barX in bars_Xmin:
                        highs_Xmin.append(barX['high'])
                        lows_Xmin.append(barX['low'])
                    highX = max(highs_Xmin)
                    lowX = min(lows_Xmin)
                    openX = bars_Xmin[0]['open']
                    closeX = bars_Xmin[len(bars_Xmin)-1]['close']
                    bar_hour = {
                        "timestamp": timestampX,
                        "time": bar['datetime'],
                        "open": openX,
                        "high": highX,
                        "low": lowX,
                        "close": closeX
                    }
                    bars_hour.append(bar_hour)
                    bars_Xmin = []
                bars_Xmin.append(bar)
            bars = bars_hour
        else:
            bars_revised = []
            bars_X = []
            highs_X = []
            lows_X = []
            counter = 0
            for bar in bars:
                timestampX = pd.to_datetime(int(bar['datetime']) * 1000000)
                if counter == factor and len(bars_X) > 0:
                    highs_X = []
                    lows_X = []
                    for barX in bars_X:
                        highs_X.append(barX['high'])
                        lows_X.append(barX['low'])
                    highX = max(highs_X)
                    lowX = min(lows_X)
                    openX = bars_X[0]['open']
                    closeX = bars_X[len(bars_X)-1]['close']
                    bar_revised = {
                        "timestamp": timestampX,
                        "time": bar['datetime'],
                        "open": openX,
                        "high": highX,
                        "low": lowX,
                        "close": closeX
                    }
                    bars_revised.append(bar_revised)
                    bars_X = []
                    counter = 0
                bars_X.append(bar)
                counter += 1
            bars = bars_revised
    return bars